In [1]:
db_id = ''
db_pw = ''
db_nm = ''

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import requests
import time
from tqdm import tqdm
import os

import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import re
import datetime
import json

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup

# DB관련 함수정의
def db_connection(host_name=host_name):
    host_url = host_url
    user_nm = db_id
    passwd = db_pw
    port_num = port_num
    db_name = db_nm
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8', cursorclass=pymysql.cursors.DictCursor)
    return conn

def engine_upload(upload_df: object, host_name: object = 'ds', db_name: object = db_name, table_name: object = table_name) -> object:
    host_url = host_url
    user_nm = db_id
    passwd = db_pw
    port_num = port_num
    db_name = db_nm
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    engine_conn = engine.connect()
    upload_df.to_sql(table_name, engine_conn, if_exists='append', index=None)
    engine_conn.close()
    engine.dispose()
    
    
directory = os.getcwd()
print("Current working directory : ", os.getcwd())

#수집날짜
folder_date = str(datetime.datetime.now().date())
folder_date = re.sub("-", ".", folder_date)
today = re.sub("\.", "", folder_date)
print('오늘 날짜 : ', folder_date)

Current working directory :  /Users/eunkk01/Desktop/goodreads
오늘 날짜 :  2023.11.20


In [8]:
conn = db_connection(host_name)
category_href = pd.read_sql(f"select * from {table_name} where status is null;", conn)
conn.close()

In [9]:
print(len(category_href))
category_href.head()

37755


,id,genre,category_list,href,count,status
0,647,art,"Kid-Friendly Drawing, Illustration, Sketching,...",https://www.goodreads.com//list/show/128067.Ki...,48,None
1,648,art,Ukrainian culture - Українська культура,https://www.goodreads.com//list/show/128164.Uk...,135,None
2,649,art,Japanese Visual Arts,https://www.goodreads.com//list/show/132188.Ja...,54,None
3,650,art,"Great Artwork, Not So Great Story",https://www.goodreads.com//list/show/126400.Gr...,35,None
4,651,art,Arts and Homosexuality,https://www.goodreads.com//list/show/127143.Ar...,58,None


In [5]:
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
for num in tqdm(range(0, len(category_href))):
    book_df = pd.DataFrame(columns = ['genre', 'category_list', 'category_rank', 'title', 'book_href', 'utm_code'])
    drv.get(category_href['href'][num])
#     drv.get('https://www.goodreads.com/list/show/21995.Best_21st_Century_Non_Fiction?page=13')

    status = True
    while status == True:
        # login popup
        try:
            drv.find_elements(By.CLASS_NAME, 'modal__close')[-1].click()
        except:
            pass

        soup = BeautifulSoup(drv.page_source, 'html.parser')
        book_list = soup.find('div', {'id':'all_votes'}).find_all('tr')

        rank = []
        title = []
        href = []
        utm_code = []

        for i in range(len(book_list)):
            rank.append(int(book_list[i].find('td', {'class':'number'}).text))
            title.append(book_list[i].find('a', {'class':'bookTitle'}).text.strip())
            get_href = book_list[i].find('a', {'class':'bookTitle'})['href']
            href.append('https://www.goodreads.com'+get_href)
            utm_code.append(int(get_href.split('/')[-1].split('-')[0].split('.')[0]))

        gr_data = [category_href['genre'][num] for g in range(len(rank))]
        cat_data = [category_href['category_list'][num] for g in range(len(rank))]

        data = pd.DataFrame({'genre':gr_data, 'category_list':cat_data, 'category_rank':rank, 
                             'title':title, 'book_href':href, 'utm_code':utm_code})
        book_df = pd.concat([book_df, data], ignore_index=True)

        try:
            try:
                if drv.find_element(By.CLASS_NAME, 'next_page.disabled').text.split(' ')[0].strip() == 'next':
                    status = False
            except:
                drv.find_element(By.CLASS_NAME, 'next_page').click()
                time.sleep(4)

        except:
            status = False
            pass
    
    # insert
    user_nm = db_id
    passwd = db_pw
    host_url = host_url
    port_num = port_num
    db_name = db_nm
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    engine_conn = engine.connect()
    
    book_df.drop_duplicates(['title', 'utm_code'], ignore_index=True, inplace=True)
    book_df.to_sql(f'{upload_table_name}', con=engine, if_exists='append', index=None)
    engine_conn.close()
    engine.dispose()
    
    # 수집한 category의 status 변경
    conn = db_connection(host_name)
    cur = conn.cursor()
    
    cur.execute(
        f'update {table_name} set status = "fin" where id = "{category_href["id"][num]}"'
    )
    conn.commit()
    
    cur.close()
    conn.close()
    print(f'{category_href["id"][num]} finished~!')
    time.sleep(1)

  0%|                                                 | 0/37755 [00:00<?, ?it/s]

In [7]:
book_df

,genre,category_list,category_rank,title,book_href,utm_code
0,art,"Art, Music, and Poetry-Inspired Cookbooks",1,Impressionist Picnics,https://www.goodreads.com/book/show/1639426.Im...,1639426
1,art,"Art, Music, and Poetry-Inspired Cookbooks",1,The Dutch Table: Gastronomy in the Golden Age ...,https://www.goodreads.com/book/show/501674.The...,501674
2,art,"Art, Music, and Poetry-Inspired Cookbooks",1,Renaissance Recipes,https://www.goodreads.com/book/show/1975468.Re...,1975468
3,art,"Art, Music, and Poetry-Inspired Cookbooks",4,"Modern Art Desserts: Recipes for Cakes, Cookie...",https://www.goodreads.com/book/show/15799042-m...,15799042
4,art,"Art, Music, and Poetry-Inspired Cookbooks",5,A feast for the eyes,https://www.goodreads.com/book/show/1301531.A_...,1301531
5,art,"Art, Music, and Poetry-Inspired Cookbooks",6,Dining with Picasso,https://www.goodreads.com/book/show/5047801-di...,5047801
6,art,"Art, Music, and Poetry-Inspired Cookbooks",7,Eat This Poem: A Literary Feast of Recipes Ins...,https://www.goodreads.com/book/show/30962132-e...,30962132
7,art,"Art, Music, and Poetry-Inspired Cookbooks",8,Cooking Out of This World,https://www.goodreads.com/book/show/177942.Coo...,177942
8,art,"Art, Music, and Poetry-Inspired Cookbooks",8,Rapper's Delight: The Hip Hop Cookbook,https://www.goodreads.com/book/show/21816171-r...,21816171
9,art,"Art, Music, and Poetry-Inspired Cookbooks",10,True Blood Cookbook,https://www.goodreads.com/book/show/15837730-t...,15837730


### 중간에 멈추면 아래 코드로 이어서

In [ ]:
drv.get('https://www.goodreads.com/list/show/21995.Best_21st_Century_Non_Fiction?page=13')

status = True
while status == True:
    # login popup
    try:
        drv.find_elements(By.CLASS_NAME, 'modal__close')[-1].click()
    except:
        pass

    soup = BeautifulSoup(drv.page_source, 'html.parser')
    book_list = soup.find('div', {'id':'all_votes'}).find_all('tr')

    rank = []
    title = []
    href = []
    utm_code = []

    for i in range(len(book_list)):
        rank.append(int(book_list[i].find('td', {'class':'number'}).text))
        title.append(book_list[i].find('a', {'class':'bookTitle'}).text.strip())
        get_href = book_list[i].find('a', {'class':'bookTitle'})['href']
        href.append('https://www.goodreads.com'+get_href)
        utm_code.append(int(get_href.split('/')[-1].split('-')[0].split('.')[0]))

    gr_data = [category_href['genre'][num] for g in range(len(rank))]
    cat_data = [category_href['category_list'][num] for g in range(len(rank))]

    data = pd.DataFrame({'genre':gr_data, 'category_list':cat_data, 'category_rank':rank, 
                         'title':title, 'book_href':href, 'utm_code':utm_code})
    book_df = pd.concat([book_df, data], ignore_index=True)

    try:
        if drv.find_element(By.CLASS_NAME, 'next_page.disabled').text.split(' ')[0].strip() == 'next':
            status = False
            print(f'{category_href["id"][num]} finished~!')
    except:
        drv.find_element(By.CLASS_NAME, 'next_page').click()
        time.sleep(4)

# 수집한 category의 status 변경
conn = db_connection('mai')
cur = conn.cursor()

cur.execute(
    f'update book_kyobo.goodreads_category_href set status = "fin" where id = "{category_href["id"][num]}"'
)
conn.commit()

cur.close()
conn.close()

user_nm = db_id
passwd = db_pw
host_url = "db.ds.mycelebs.com"
port_num = 3306
db_name = db_nm
engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
engine_conn = engine.connect()

# insert
book_df.to_sql(f'goodreads_book_href', con=engine, if_exists='append', index=None)
engine_conn.close()
engine.dispose()